<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [1]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime
from pandas import json_normalize 
# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [25]:
def getBoosterVersion(data):
    booster_versions = []
    for rocket_id in data['rocket']:
        response = requests.get(f"https://api.spacexdata.com/v4/rockets/{rocket_id}").json()
        booster_versions.append(response['name'])
    return booster_versions

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [24]:
def getLaunchSite(data):
    launch_sites = []
    longitudes = []
    latitudes = []
    for launchpad_id in data['launchpad']:
        response = requests.get(f"https://api.spacexdata.com/v4/launchpads/{launchpad_id}").json()
        launch_sites.append(response['name'])
        longitudes.append(response['longitude'])
        latitudes.append(response['latitude'])
    return launch_sites, longitudes, latitudes

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [38]:
def getPayloadData(data):
    payload_masses = []
    orbits = []
    for payload_id_list in data['payloads']:
        # Assuming payload_id_list is a list of payload IDs
        payload_id = payload_id_list[0]  # Use the first payload if multiple are present
        response = requests.get(f"https://api.spacexdata.com/v4/payloads/{payload_id}").json()
        payload_masses.append(response.get('mass_kg', None))
        orbits.append(response.get('orbit', None))
    return payload_masses, orbits


From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [22]:
def getCoreData(data):
    outcomes = []
    landing_types = []
    flight_counts = []
    gridfins_used = []
    reused_cores = []
    legs_used = []
    landing_pads = []
    blocks = []
    reused_counts = []
    serials = []
    
    for core_list in data['cores']:
        core = core_list[0]  # Use the first core if multiple are present
        if core['core'] is not None:
            response = requests.get(f"https://api.spacexdata.com/v4/cores/{core['core']}").json()
            blocks.append(response.get('block', None))
            reused_counts.append(response.get('reuse_count', None))
            serials.append(response.get('serial', None))
        else:
            blocks.append(None)
            reused_counts.append(None)
            serials.append(None)
        
        outcomes.append(f"{core['landing_success']} {core.get('landing_type', 'Unknown')}")
        flight_counts.append(core['flight'])
        gridfins_used.append(core['gridfins'])
        reused_cores.append(core['reused'])
        legs_used.append(core['legs'])
        landing_pads.append(core['landpad'])
    
    return outcomes, landing_types, flight_counts, gridfins_used, reused_cores, legs_used, landing_pads, blocks, reused_counts, serials


Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [6]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [7]:
response = requests.get(spacex_url)

Check the content of the response


NameError: name 'hello' is not defined

You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [8]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [9]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [26]:
# Step 1: Make the API request and decode the JSON content
spacex_url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)

# Check if the request was successful
if response.status_code == 200:
    # Decode the JSON content into a Python object (list of dictionaries)
    json_data = response.json()
    
    # Step 2: Normalize the JSON data into a flat DataFrame
    data = pd.json_normalize(json_data)
    
    # Display the first few rows of the DataFrame
    print(data.head())
else:
    print("Failed to retrieve data from SpaceX API")

       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                                                                                            failures  \
0                                                [{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]   
1            [{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutd

Using the dataframe <code>data</code> print the first 5 rows


In [27]:
data.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [28]:
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']] 

# Remove rows with multiple cores and multiple payloads 
data = data[data['cores'].map(len) == 1] 
data = data[data['payloads'].map(len) == 1]

# Extract the single value in the list and replace the feature 
data['cores'] = data['cores'].map(lambda x: x[0]) 
data['payloads'] = data['payloads'].map(lambda x: x[0]) 

# Convert the date_utc to a datetime datatype and extract the date 
data['date'] = pd.to_datetime(data['date_utc']).dt.date 

# Restrict the dates of the launches 
data = data[data['date'] <= datetime.date(2020, 11, 13)]

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* **From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.**

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [29]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [30]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [31]:
getBoosterVersion(data)

['Falcon 1',
 'Falcon 1',
 'Falcon 1',
 'Falcon 1',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',
 'Falcon 9',

the list has now been update 


In [33]:
BoosterVersion[0:5]

[]

we can apply the rest of the  functions here:


In [34]:
# Call getLaunchSite
getLaunchSite(data)

(['Kwajalein Atoll',
  'Kwajalein Atoll',
  'Kwajalein Atoll',
  'Kwajalein Atoll',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'VAFB SLC 4E',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'VAFB SLC 4E',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'VAFB SLC 4E',
  'KSC LC 39A',
  'KSC LC 39A',
  'KSC LC 39A',
  'KSC LC 39A',
  'KSC LC 39A',
  'KSC LC 39A',
  'KSC LC 39A',
  'VAFB SLC 4E',
  'KSC LC 39A',
  'KSC LC 39A',
  'VAFB SLC 4E',
  'KSC LC 39A',
  'VAFB SLC 4E',
  'KSC LC 39A',
  'KSC LC 39A',
  'CCSFS SLC 40',
  'VAFB SLC 4E',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'VAFB SLC 4E',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'KSC LC 39A',
  'CCSFS SLC 40',
  'CCSFS SLC 40',
  'CCSFS SLC 4

In [39]:
# Call getPayloadData
getPayloadData(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [40]:
# Call getCoreData
getCoreData(data)

KeyError: 0

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [43]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [45]:
# Find the maximum length
max_length = max(len(lst) for lst in launch_dict.values())

# Pad shorter lists with None
for key, value in launch_dict.items():
    if len(value) < max_length:
        launch_dict[key].extend([None] * (max_length - len(value)))

# Now create the DataFrame
launch_df = pd.DataFrame(launch_dict)


Show the summary of the dataframe


In [46]:
# Show the head of the dataframe
print(launch_df.head())


   FlightNumber        Date BoosterVersion PayloadMass Orbit LaunchSite  \
0             1  2006-03-24           None        None  None       None   
1             2  2007-03-21           None        None  None       None   
2             4  2008-09-28           None        None  None       None   
3             5  2009-07-13           None        None  None       None   
4             6  2010-06-04           None        None  None       None   

  Outcome Flights GridFins Reused  Legs LandingPad Block ReusedCount Serial  \
0    None    None     None   None  None       None  None        None   None   
1    None    None     None   None  None       None  None        None   None   
2    None    None     None   None  None       None  None        None   None   
3    None    None     None   None  None       None  None        None   None   
4    None    None     None   None  None       None  None        None   None   

  Longitude Latitude  
0      None     None  
1      None     None  
2    

### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [47]:
# Hint data['BoosterVersion']!='Falcon 1'
# Filter the DataFrame to only include rows where the BoosterVersion is not 'Falcon 1'
data_falcon9 = launch_df[launch_df['BoosterVersion'] != 'Falcon 1']

# Optionally, reset the index of the filtered DataFrame (if you want a clean index)
data_falcon9.reset_index(drop=True, inplace=True)

# Display the first few rows of the filtered DataFrame
print(data_falcon9.head())

   FlightNumber        Date BoosterVersion PayloadMass Orbit LaunchSite  \
0             1  2006-03-24           None        None  None       None   
1             2  2007-03-21           None        None  None       None   
2             4  2008-09-28           None        None  None       None   
3             5  2009-07-13           None        None  None       None   
4             6  2010-06-04           None        None  None       None   

  Outcome Flights GridFins Reused  Legs LandingPad Block ReusedCount Serial  \
0    None    None     None   None  None       None  None        None   None   
1    None    None     None   None  None       None  None        None   None   
2    None    None     None   None  None       None  None        None   None   
3    None    None     None   None  None       None  None        None   None   
4    None    None     None   None  None       None  None        None   None   

  Longitude Latitude  
0      None     None  
1      None     None  
2    

Now that we have removed some values we should reset the FlgihtNumber column


In [48]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2,2007-03-21,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,3,2008-09-28,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,4,2009-07-13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,5,2010-06-04,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,90,2020-09-03,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
90,91,2020-10-06,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
91,92,2020-10-18,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
92,93,2020-10-24,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [49]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion    94
PayloadMass       94
Orbit             94
LaunchSite        94
Outcome           94
Flights           94
GridFins          94
Reused            94
Legs              94
LandingPad        94
Block             94
ReusedCount       94
Serial            94
Longitude         94
Latitude          94
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [50]:
# Step 1: Calculate the mean value of the PayloadMass column, ignoring NaN values
payload_mass_mean = data_falcon9['PayloadMass'].mean()

# Step 2: Replace the np.nan values in the PayloadMass column with the mean value
data_falcon9['PayloadMass'].replace(np.nan, payload_mass_mean, inplace=True)


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


<!--## Change Log
-->


<!--

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|
-->


Copyright © 2021 IBM Corporation. All rights reserved.
